# Data Analysis and Pipeline Verification

- This notebook analyzes the flight price prediction dataset at different stages of the MLOps pipeline: raw, bronze, silver, and gold. The goal is to perform basic data analysis at each stage and verify if the corresponding data processing pipelines have worked as expected.
- Gets the test instances utilized for local explanation by shapley

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

## 1. Raw Data Analysis

We start by loading the raw dataset and performing a preliminary analysis.

In [2]:
from shared.config import core_paths, config_bronze, config_gold, config_silver

In [3]:
raw_df = pd.read_csv(core_paths.RAW_DATA_DIR / "train_validation_test" / "train.csv")
raw_df.head()

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date
0,0,0,Recife (PE),Florianopolis (SC),firstClass,1434.38,1.76,676.53,FlyingDrops,2019-09-26
1,121138,1202,Florianopolis (SC),Natal (RN),firstClass,1315.27,1.84,709.37,CloudFy,2019-09-26
2,132076,1301,Florianopolis (SC),Salvador (BH),premium,1311.38,2.44,937.77,CloudFy,2019-09-26
3,28904,276,Recife (PE),Rio de Janeiro (RJ),economic,908.93,2.30,885.57,Rainbow,2019-09-26
4,88695,877,Aracaju (SE),Natal (RN),firstClass,598.61,0.46,176.33,CloudFy,2019-09-26


In [4]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190321 entries, 0 to 190320
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   travelCode  190321 non-null  int64  
 1   userCode    190321 non-null  int64  
 2   from        190321 non-null  object 
 3   to          190321 non-null  object 
 4   flightType  190321 non-null  object 
 5   price       190321 non-null  float64
 6   time        190321 non-null  float64
 7   distance    190321 non-null  float64
 8   agency      190321 non-null  object 
 9   date        190321 non-null  object 
dtypes: float64(3), int64(2), object(5)
memory usage: 14.5+ MB


In [5]:
raw_df.describe()

,travelCode,userCode,price,time,distance
count,190321.000000,190321.000000,190321.000000,190321.000000,190321.000000
mean,67919.412272,667.159567,955.601391,1.418343,545.875221
std,39051.352769,387.624777,362.432241,0.543563,209.246707
min,0.000000,0.000000,301.510000,0.440000,168.220000
25%,34245.000000,328.000000,672.660000,1.020000,392.760000
50%,68332.000000,662.000000,899.600000,1.460000,562.140000
75%,101679.000000,1009.000000,1222.240000,1.760000,676.530000
max,135943.000000,1339.000000,1754.170000,2.440000,937.770000


In [6]:
raw_df.describe(include="object")

,from,to,flightType,agency,date
count,190321,190321,190321,190321,190321
unique,9,9,3,3,461
top,Florianopolis (SC),Florianopolis (SC),firstClass,Rainbow,2019-09-26
freq,39868,39737,81242,81741,1335


In [7]:
raw_df.nunique()

travelCode    95430
userCode       1335
from              9
to                9
flightType        3
price           490
time             33
distance         35
agency            3
date            461
dtype: int64

In [8]:
cat_cols = list(raw_df.select_dtypes(include="object").columns)
cat_cols.remove("date")
for col in cat_cols:
    unique_cats = list(raw_df[col].unique())
    print(f"{col}:{unique_cats}")

from:['Recife (PE)', 'Florianopolis (SC)', 'Aracaju (SE)', 'Campo Grande (MS)', 'Brasilia (DF)', 'Natal (RN)', 'Sao Paulo (SP)', 'Rio de Janeiro (RJ)', 'Salvador (BH)']
to:['Florianopolis (SC)', 'Natal (RN)', 'Salvador (BH)', 'Rio de Janeiro (RJ)', 'Aracaju (SE)', 'Sao Paulo (SP)', 'Recife (PE)', 'Campo Grande (MS)', 'Brasilia (DF)']
flightType:['firstClass', 'premium', 'economic']
agency:['FlyingDrops', 'CloudFy', 'Rainbow']


### Initial Thoughts on Raw Data:
- The dataset contains a mix of numerical and categorical features.
- Column names are not standardized.


## 2. Bronze Data Analysis

In [9]:
bronze_train_df = pd.read_csv(config_bronze.BRONZE_PROCESSED_DIR / "train.csv")
bronze_train_df.head()

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date
0,0,0,Recife (PE),Florianopolis (SC),firstClass,1434.38,1.76,676.53,FlyingDrops,2019-09-26
1,121138,1202,Florianopolis (SC),Natal (RN),firstClass,1315.27,1.84,709.37,CloudFy,2019-09-26
2,132076,1301,Florianopolis (SC),Salvador (BH),premium,1311.38,2.44,937.77,CloudFy,2019-09-26
3,28904,276,Recife (PE),Rio de Janeiro (RJ),economic,908.93,2.30,885.57,Rainbow,2019-09-26
4,88695,877,Aracaju (SE),Natal (RN),firstClass,598.61,0.46,176.33,CloudFy,2019-09-26


In [10]:
bronze_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190321 entries, 0 to 190320
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   travelCode  190321 non-null  int64  
 1   userCode    190321 non-null  int64  
 2   from        190321 non-null  object 
 3   to          190321 non-null  object 
 4   flightType  190321 non-null  object 
 5   price       190321 non-null  float64
 6   time        190321 non-null  float64
 7   distance    190321 non-null  float64
 8   agency      190321 non-null  object 
 9   date        190321 non-null  object 
dtypes: float64(3), int64(2), object(5)
memory usage: 14.5+ MB


In [11]:
bronze_train_df.describe()

,travelCode,userCode,price,time,distance
count,190321.000000,190321.000000,190321.000000,190321.000000,190321.000000
mean,67919.412272,667.159567,955.601391,1.418343,545.875221
std,39051.352769,387.624777,362.432241,0.543563,209.246707
min,0.000000,0.000000,301.510000,0.440000,168.220000
25%,34245.000000,328.000000,672.660000,1.020000,392.760000
50%,68332.000000,662.000000,899.600000,1.460000,562.140000
75%,101679.000000,1009.000000,1222.240000,1.760000,676.530000
max,135943.000000,1339.000000,1754.170000,2.440000,937.770000


In [12]:
bronze_train_df.describe(include="object")

,from,to,flightType,agency,date
count,190321,190321,190321,190321,190321
unique,9,9,3,3,461
top,Florianopolis (SC),Florianopolis (SC),firstClass,Rainbow,2019-09-26
freq,39868,39737,81242,81741,1335


In [13]:
bronze_train_df.nunique()

travelCode    95430
userCode       1335
from              9
to                9
flightType        3
price           490
time             33
distance         35
agency            3
date            461
dtype: int64

### Bronze Pipeline Verification
The `bronze_pipeline.py` is mainly responsible for data validation. It checks for:
- Column presence and order.
- Data types.
Since the bronze data is just a validated version of the raw data, we don't expect to see significant changes in the data itself, but we can be more confident in its quality.

## 3. Silver Data Analysis

In [14]:
silver_train_df = pd.read_parquet(config_silver.SILVER_PROCESSED_DIR / "train.parquet")
silver_train_df.head()

,travel_code,user_code,from_location,to_location,flight_type,price,time,distance,agency,date,year,month,day,day_of_week,day_of_year,week_of_year
0,0,0,Recife (PE),Florianopolis (SC),firstClass,1434.380005,1.76,676.530029,FlyingDrops,2019-09-26,2019,9,26,3,269,39
1,121138,1202,Florianopolis (SC),Natal (RN),firstClass,1315.270020,1.84,709.369995,CloudFy,2019-09-26,2019,9,26,3,269,39
2,132076,1301,Florianopolis (SC),Salvador (BH),premium,1311.380005,2.44,937.770020,CloudFy,2019-09-26,2019,9,26,3,269,39
3,28904,276,Recife (PE),Rio de Janeiro (RJ),economic,908.929993,2.30,885.570007,Rainbow,2019-09-26,2019,9,26,3,269,39
4,88695,877,Aracaju (SE),Natal (RN),firstClass,598.609985,0.46,176.330002,CloudFy,2019-09-26,2019,9,26,3,269,39


In [15]:
silver_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190321 entries, 0 to 190320
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   travel_code    190321 non-null  int32         
 1   user_code      190321 non-null  int16         
 2   from_location  190321 non-null  category      
 3   to_location    190321 non-null  category      
 4   flight_type    190321 non-null  category      
 5   price          190321 non-null  float32       
 6   time           190321 non-null  float32       
 7   distance       190321 non-null  float32       
 8   agency         190321 non-null  category      
 9   date           190321 non-null  datetime64[ns]
 10  year           190321 non-null  int16         
 11  month          190321 non-null  int8          
 12  day            190321 non-null  int8          
 13  day_of_week    190321 non-null  int8          
 14  day_of_year    190321 non-null  int16         
 15  

In [16]:
silver_train_df.describe()

,travel_code,user_code,price,time,distance,date,year,month,day,day_of_week,day_of_year,week_of_year
count,190321.000000,190321.000000,190321.000000,190321.000000,190321.000000,190321,190321.000000,190321.000000,190321.000000,190321.000000,190321.000000,190321.000000
mean,67919.412272,667.159567,955.601379,1.418343,545.875244,2020-07-11 06:14:53.299215872,2020.032041,6.449950,15.853080,3.373122,181.345238,26.484939
min,0.000000,0.000000,301.510010,0.440000,168.220001,2019-09-26 00:00:00,2019.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,34245.000000,328.000000,672.659973,1.020000,392.760010,2020-02-02 00:00:00,2020.000000,3.000000,8.000000,3.000000,80.000000,12.000000
50%,68332.000000,662.000000,899.599976,1.460000,562.140015,2020-06-26 00:00:00,2020.000000,6.000000,16.000000,3.000000,168.000000,24.000000
75%,101679.000000,1009.000000,1222.239990,1.760000,676.530029,2020-12-10 00:00:00,2020.000000,10.000000,24.000000,4.000000,290.000000,42.000000
max,135943.000000,1339.000000,1754.170044,2.440000,937.770020,2021-07-01 00:00:00,2021.000000,12.000000,31.000000,6.000000,366.000000,53.000000
std,39051.352769,387.624777,362.426453,0.543455,209.208664,NaN,0.638354,3.673053,8.843703,1.653325,112.403690,16.158775


In [17]:
silver_train_df.describe(include=["object", "category"])

,from_location,to_location,flight_type,agency
count,190321,190321,190321,190321
unique,9,9,3,3
top,Florianopolis (SC),Florianopolis (SC),firstClass,Rainbow
freq,39868,39737,81242,81741


In [18]:
cat_silver_df = silver_train_df.select_dtypes(include=["category"])
cat_silver_df.columns

Index(['from_location', 'to_location', 'flight_type', 'agency'], dtype='object')

In [19]:
for col in cat_silver_df.columns:
    print(f"{cat_silver_df[col].cat.categories.to_list()}")

['Aracaju (SE)', 'Brasilia (DF)', 'Campo Grande (MS)', 'Florianopolis (SC)', 'Natal (RN)', 'Recife (PE)', 'Rio de Janeiro (RJ)', 'Salvador (BH)', 'Sao Paulo (SP)']
['Aracaju (SE)', 'Brasilia (DF)', 'Campo Grande (MS)', 'Florianopolis (SC)', 'Natal (RN)', 'Recife (PE)', 'Rio de Janeiro (RJ)', 'Salvador (BH)', 'Sao Paulo (SP)']
['economic', 'firstClass', 'premium']
['CloudFy', 'FlyingDrops', 'Rainbow']


In [20]:
silver_train_df.nunique()

travel_code      95430
user_code         1335
from_location        9
to_location          9
flight_type          3
price              490
time                33
distance            35
agency               3
date               461
year                 3
month               12
day                 31
day_of_week          5
day_of_year        327
week_of_year        53
dtype: int64

In [21]:
silver_train_df.duplicated(keep=False).sum()

0

In [22]:
silver_train_dropped = silver_train_df.drop(
    columns=[
        "travel_code",
        "user_code",
        "date",
        "day_of_year",
        "week_of_year",
    ]
)

In [23]:
silver_train_dropped.sample(5)

,from_location,to_location,flight_type,price,time,distance,agency,year,month,day,day_of_week
178241,Florianopolis (SC),Salvador (BH),firstClass,1630.750000,2.44,937.770020,Rainbow,2021,5,6,3
120626,Recife (PE),Natal (RN),economic,360.220001,0.58,222.669998,Rainbow,2020,9,19,5
86305,Florianopolis (SC),Sao Paulo (SP),premium,554.869995,1.46,562.140015,CloudFy,2020,5,28,3
165873,Salvador (BH),Florianopolis (SC),economic,899.599976,2.44,937.770020,Rainbow,2021,3,12,4
93011,Recife (PE),Florianopolis (SC),firstClass,1371.829956,1.76,676.530029,Rainbow,2020,6,19,4


In [24]:
print(len(silver_train_df))
print(len(silver_train_dropped))

190321
190321


In [25]:
silver_train_dropped.duplicated(keep="first").sum()

95579

In [26]:
silver_test_df = pd.read_parquet(config_silver.SILVER_PROCESSED_DIR / "test.parquet")
silver_test_dropped = silver_train_df.drop(
    columns=[
        "travel_code",
        "user_code",
        "date",
        "day_of_year",
        "week_of_year",
    ]
)

### Silver Pipeline Verification
The `silver_pipeline.py` performs several transformations:
- **Column Renaming and Standardization**: Column names are now in snake_case.
- **Data Type Optimization**: Data types have been optimized (e.g., `date` is now a datetime object).
- **Feature Engineering**: New date-related features have been created (`year`, `month`, `day`, etc.).
- **Duplicate Handling**: Erroneous duplicates have been removed.
By comparing the silver data to the bronze data, we can verify that these transformations have been applied correctly.

## 4. Gold Data Analysis

In [27]:
gold_train_df = pd.read_parquet(config_gold.GOLD_PROCESSED_DIR / "train.parquet")
gold_train_df.head()

,flight_type,from_location,to_location,agency,price,time,distance,year,month_sin,month_cos,day_of_week_sin,day_of_week_cos,day_sin,day_cos
0,2.0,5.0,3.0,1.0,1434.380005,1.76,676.530029,2019.0,-1.0,-1.836970e-16,0.433884,-0.900969,-0.848644,0.528964
1,2.0,3.0,4.0,0.0,1315.270020,1.84,709.369995,2019.0,-1.0,-1.836970e-16,0.433884,-0.900969,-0.848644,0.528964
2,1.0,3.0,7.0,0.0,1311.380005,2.44,937.770020,2019.0,-1.0,-1.836970e-16,0.433884,-0.900969,-0.848644,0.528964
3,0.0,5.0,6.0,2.0,908.929993,2.30,885.570007,2019.0,-1.0,-1.836970e-16,0.433884,-0.900969,-0.848644,0.528964
4,2.0,0.0,4.0,0.0,598.609985,0.46,176.330002,2019.0,-1.0,-1.836970e-16,0.433884,-0.900969,-0.848644,0.528964


In [28]:
gold_train_df.sample(50).to_csv("gold_sample.csv")

In [29]:
gold_train_df.columns

Index(['flight_type', 'from_location', 'to_location', 'agency', 'price',
       'time', 'distance', 'year', 'month_sin', 'month_cos', 'day_of_week_sin',
       'day_of_week_cos', 'day_sin', 'day_cos'],
      dtype='object')

In [30]:
gold_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94742 entries, 0 to 94741
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   flight_type      94742 non-null  float64
 1   from_location    94742 non-null  float64
 2   to_location      94742 non-null  float64
 3   agency           94742 non-null  float64
 4   price            94742 non-null  float64
 5   time             94742 non-null  float64
 6   distance         94742 non-null  float64
 7   year             94742 non-null  float64
 8   month_sin        94742 non-null  float64
 9   month_cos        94742 non-null  float64
 10  day_of_week_sin  94742 non-null  float64
 11  day_of_week_cos  94742 non-null  float64
 12  day_sin          94742 non-null  float64
 13  day_cos          94742 non-null  float64
dtypes: float64(14)
memory usage: 10.1 MB


In [31]:
gold_train_df.describe()

,flight_type,from_location,to_location,agency,price,time,distance,year,month_sin,month_cos,day_of_week_sin,day_of_week_cos,day_sin,day_cos
count,94742.000000,94742.000000,94742.000000,94742.000000,94742.000000,94742.000000,94742.000000,94742.000000,9.474200e+04,9.474200e+04,94742.000000,94742.000000,9.474200e+04,94742.000000
mean,1.142249,3.785956,3.209960,1.000158,944.993863,1.351191,520.040287,2020.077484,8.310325e-02,8.454659e-02,-0.228624,-0.208192,-3.698261e-03,-0.009739
std,0.833229,2.575361,2.393595,0.925617,357.578769,0.548807,211.302136,0.642241,6.883345e-01,7.156478e-01,0.552117,0.774315,7.128550e-01,0.701242
min,0.000000,0.000000,0.000000,0.000000,301.510010,0.440000,168.220001,2019.000000,-1.000000e+00,-1.000000e+00,-0.974928,-0.900969,-9.987165e-01,-0.994869
25%,0.000000,2.000000,1.000000,0.000000,667.000000,0.850000,327.549988,2020.000000,-5.000000e-01,-5.000000e-01,-0.781831,-0.900969,-7.247928e-01,-0.758758
50%,1.000000,4.000000,3.000000,1.000000,889.070007,1.440000,555.739990,2020.000000,1.224647e-16,6.123234e-17,-0.433884,-0.222521,-2.449294e-16,-0.050649
75%,2.000000,6.000000,5.000000,2.000000,1209.040039,1.760000,676.530029,2020.000000,8.660254e-01,8.660254e-01,0.433884,0.623490,7.247928e-01,0.688967
max,2.000000,8.000000,8.000000,2.000000,1754.170044,2.440000,937.770020,2021.000000,1.000000e+00,1.000000e+00,0.433884,1.000000,9.987165e-01,1.000000


In [32]:
gold_train_df.nunique()

flight_type          3
from_location        9
to_location          9
agency               3
price              490
time                33
distance            35
year                 3
month_sin           11
month_cos           11
day_of_week_sin      5
day_of_week_cos      5
day_sin             31
day_cos             26
dtype: int64

### Gold Pipeline Verification
The `gold_pipeline.py` applies the final feature engineering and preprocessing steps:
- **Duplcates**: after dropping unique identification columns(travel_code and user_code) there were no erroneous duplicates but duplicate data due to different people boarding the same flight for the same route, these duplicate rows were droped successfully.
- **Imputation**: Missing values have been imputed.
- **Feature Engineering**: Cyclical features and interaction features have been created.
- **Categorical Encoding**: Categorical features have been encoded.
- **Outlier Handling**: Outliers have been handled.
- **Power Transformation and Scaling**: Numerical features have been transformed and scaled.
The gold data is now ready for model training. We can see that all columns are numerical and there are no missing values.

## Get the test indices that were tested with SHAP for prediction by Lightgbm champion model

In [33]:
test_gold = pd.read_parquet(config_gold.GOLD_PROCESSED_DIR / "test.parquet")

follow the same sampling method with random state used in the shap_explain.py
we used 2000 samples with random_state 42 and the first three instances for that.

In [34]:
# shap_local_instances = test_gold.sample(n=2000, random_state=42).head(3)

In [35]:
# shap_local_instances

lets get the same instances in raw from the silver dataset so we can make sense of the predictions and features.

In [36]:
# shap_local_instances.to_csv(
#     core_paths.DOCS_DIR / "Final_model_shap_local_instances.csv"
# )
# shap_local_instances_unprocessed.to_csv(
#     core_paths.DOCS_DIR / "shap_local_instances_unprocessed.csv"
# )

### Checking Prediction Server



In [37]:
from dotenv import load_dotenv
from prediction_server.model_loader import (
    load_production_model,
    load_preprocessing_artifacts,
)
from prediction_server.predict import preprocessing_for_prediction
from prediction_server import config_app

In [38]:
load_dotenv()

True

In [39]:
model = load_production_model(
    config_app.MODEL_NAME,
    config_app.MODEL_VERSION_ALIAS,
    model_flavor=config_app.MODEL_FLAVOR,
)

In [40]:
instance = np.array([test_gold.drop(columns=["price"]).iloc[0]])
instance.shape

(1, 13)

In [41]:
actual_price = test_gold["price"][0]
actual_price = f"{actual_price:.2f}"

In [42]:
predicted_price = model.predict(instance)[0]
predicted_price = f"{predicted_price:.2f}"
print(f"predicted_price:{predicted_price}, actual_price:{actual_price}")

predicted_price:696.72, actual_price:697.22


In [43]:
raw_instance = raw_df.drop(columns=["price"]).iloc[[0]]
raw_instance

,travelCode,userCode,from,to,flightType,time,distance,agency,date
0,0,0,Recife (PE),Florianopolis (SC),firstClass,1.76,676.53,FlyingDrops,2019-09-26


In [50]:
preprocessors = load_preprocessing_artifacts()
preprocessed_df = preprocessing_for_prediction(raw_instance, preprocessors)

Column 'price' was not found in DataFrame during transform. Skipping.
Target column 'price' not found in CategoricalEncoder. Adding a dummy column for transformation.


In [51]:
preprocessed_df

,flight_type,from_location,to_location,agency,time,distance,year,month_sin,month_cos,day_of_week_sin,day_of_week_cos,day_sin,day_cos
0,2.0,5.0,3.0,1.0,1.76,676.53,2019.0,-1.0,-1.836970e-16,0.433884,-0.900969,-0.848644,0.528964


In [47]:
actual_price = raw_df["price"][0]

In [48]:
prediction_price = model.predict(preprocessed_df)[0]

In [49]:
prediction_price = f"{prediction_price:.2f}"
print(f"predicted_price:{prediction_price}, actual_price:{actual_price}")

predicted_price:1434.35, actual_price:1434.38


In [ ]:
# sample
{
    "from_location": "Recife (PE)",
    "to_location": "Florianopolis (SC)",
    "flight_type": "firstClass",
    "time": 1.76,
    "distance": 676.53,
    "agency": "FlyingDrops",
    "date": "2019-09-26",
}

{'from_location': 'Recife (PE)',
 'to_location': 'Florianopolis (SC)',
 'flight_type': 'firstClass',
 'time': 1.76,
 'distance': 676.53,
 'agency': 'FlyingDrops',
 'date': '2019-09-26'}